<a href="https://colab.research.google.com/github/Andyqballer/Image-Resolution/blob/main/MidSemHomeWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torchvision.utils as vutils
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tqdm import tqdm
import time
import gc

class RetinalDataset(Dataset):
    def __init__(self, root_dir, transform=None, hr_size=128, lr_size=32):
        self.root_dir = root_dir
        self.transform = transform
        self.hr_size = hr_size
        self.lr_size = lr_size
        self.classes = ['DME', 'DRUSEN']

        self.image_paths = []
        self.labels = []

        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path, img_name))
                self.labels.append(class_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            hr_image = self.transform(image)

        lr_transform = transforms.Compose([
            transforms.Resize((self.lr_size, self.lr_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        lr_image = lr_transform(image)

        return {'lr': lr_image, 'hr': hr_image, 'label': torch.tensor(label, dtype=torch.long)}

def show_transformed_samples(dataset, num_samples=5, save_path='transformed_samples.png'):
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 6))

    for i in range(num_samples):
        sample = dataset[i]

        # Show LR image
        axes[0, i].imshow(sample['lr'].permute(1, 2, 0).numpy())
        axes[0, i].set_title('LR (32x32)')
        axes[0, i].axis('off')

        # Show HR image
        axes[1, i].imshow(sample['hr'].permute(1, 2, 0).numpy())
        axes[1, i].set_title('HR (128x128)')
        axes[1, i].axis('off')

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

class MetricsTracker:
    def __init__(self):
        self.metrics = {
            'g_losses': [],
            'd_losses': [],
            'cls_a_losses': [],
            'cls_b_losses': [],
            'cls_a_acc': [],
            'cls_b_acc': []
        }

    def update(self, g_loss, d_loss, cls_a_loss, cls_b_loss, cls_a_acc, cls_b_acc):
        self.metrics['g_losses'].append(g_loss)
        self.metrics['d_losses'].append(d_loss)
        self.metrics['cls_a_losses'].append(cls_a_loss)
        self.metrics['cls_b_losses'].append(cls_b_loss)
        self.metrics['cls_a_acc'].append(cls_a_acc)
        self.metrics['cls_b_acc'].append(cls_b_acc)

    def plot_metrics(self, save_path='training_metrics.png'):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

        # Plot losses
        ax1.plot(self.metrics['g_losses'], label='Generator Loss')
        ax1.plot(self.metrics['d_losses'], label='Discriminator Loss')
        ax1.plot(self.metrics['cls_a_losses'], label='Classifier A Loss')
        ax1.plot(self.metrics['cls_b_losses'], label='Classifier B Loss')
        ax1.set_title('Training Losses')
        ax1.set_xlabel('Iteration')
        ax1.set_ylabel('Loss')
        ax1.legend()

        # Plot accuracies
        ax2.plot(self.metrics['cls_a_acc'], label='Classifier A Accuracy')
        ax2.plot(self.metrics['cls_b_acc'], label='Classifier B Accuracy')
        ax2.set_title('Classification Accuracy')
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Accuracy')
        ax2.legend()

        plt.tight_layout()
        plt.savefig(save_path)
        plt.close()

def train_models(train_loader, val_loader, models, optimizers, criteria, device,
                num_epochs=150, save_interval=10, checkpoint_dir='checkpoints'):
    os.makedirs(checkpoint_dir, exist_ok=True)
    generator, discriminator, classifier_a, classifier_b = models
    optimizer_g, optimizer_d, optimizer_cls_a, optimizer_cls_b = optimizers
    criterion_gan, criterion_content, criterion_cls = criteria

    metrics_tracker = MetricsTracker()
    best_val_acc = 0.0
    start_time = time.time()

    for epoch in range(num_epochs):
        # Training loop with progress bar
        generator.train()
        discriminator.train()
        classifier_a.train()
        classifier_b.train()

        running_g_loss = 0.0
        running_d_loss = 0.0
        running_cls_a_loss = 0.0
        running_cls_b_loss = 0.0
        running_cls_a_correct = 0
        running_cls_b_correct = 0
        total_samples = 0

        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        for batch in pbar:
            lr_images = batch['lr'].to(device)
            hr_images = batch['hr'].to(device)
            labels = batch['label'].to(device)
            batch_size = lr_images.size(0)
            total_samples += batch_size

            # Train Discriminator
            optimizer_d.zero_grad()
            generated_images = generator(lr_images)
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            d_real_loss = criterion_gan(discriminator(hr_images), real_labels)
            d_fake_loss = criterion_gan(discriminator(generated_images.detach()), fake_labels)
            d_loss = (d_real_loss + d_fake_loss) / 2
            d_loss.backward()
            optimizer_d.step()

            # Train Generator
            optimizer_g.zero_grad()
            g_content_loss = criterion_content(generated_images, hr_images)
            g_adv_loss = criterion_gan(discriminator(generated_images), real_labels)
            g_loss = g_content_loss + 0.001 * g_adv_loss
            g_loss.backward()
            optimizer_g.step()

            # Train Classifier A
            optimizer_cls_a.zero_grad()
            outputs_a = classifier_a(hr_images)
            loss_a = criterion_cls(outputs_a, labels)
            loss_a.backward()
            optimizer_cls_a.step()

            # Train Classifier B
            optimizer_cls_b.zero_grad()
            outputs_b = classifier_b(generated_images.detach())
            loss_b = criterion_cls(outputs_b, labels)
            loss_b.backward()
            optimizer_cls_b.step()

            # Calculate accuracies
            _, preds_a = torch.max(outputs_a, 1)
            _, preds_b = torch.max(outputs_b, 1)
            running_cls_a_correct += (preds_a == labels).sum().item()
            running_cls_b_correct += (preds_b == labels).sum().item()

            # Update running losses
            running_g_loss += g_loss.item()
            running_d_loss += d_loss.item()
            running_cls_a_loss += loss_a.item()
            running_cls_b_loss += loss_b.item()

            # Update progress bar
            pbar.set_postfix({
                'G_loss': g_loss.item(),
                'D_loss': d_loss.item(),
                'ClsA_loss': loss_a.item(),
                'ClsB_loss': loss_b.item()
            })

            # Free up memory
            del generated_images, outputs_a, outputs_b
            torch.cuda.empty_cache()

        # Calculate epoch metrics
        epoch_g_loss = running_g_loss / len(train_loader)
        epoch_d_loss = running_d_loss / len(train_loader)
        epoch_cls_a_loss = running_cls_a_loss / len(train_loader)
        epoch_cls_b_loss = running_cls_b_loss / len(train_loader)
        epoch_cls_a_acc = running_cls_a_correct / total_samples
        epoch_cls_b_acc = running_cls_b_correct / total_samples

        # Update metrics tracker
        metrics_tracker.update(
            epoch_g_loss, epoch_d_loss,
            epoch_cls_a_loss, epoch_cls_b_loss,
            epoch_cls_a_acc, epoch_cls_b_acc
        )

        # Save checkpoint
        if (epoch + 1) % save_interval == 0:
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pth')
            torch.save({
                'epoch': epoch,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'classifier_a_state_dict': classifier_a.state_dict(),
                'classifier_b_state_dict': classifier_b.state_dict(),
                'optimizer_g_state_dict': optimizer_g.state_dict(),
                'optimizer_d_state_dict': optimizer_d.state_dict(),
                'optimizer_cls_a_state_dict': optimizer_cls_a.state_dict(),
                'optimizer_cls_b_state_dict': optimizer_cls_b.state_dict(),
                'metrics': metrics_tracker.metrics
            }, checkpoint_path)

            # Save sample images
            save_sample_images(val_loader, generator, device,
                             save_path=f'samples_epoch_{epoch+1}.png')

        # Plot and save metrics
        metrics_tracker.plot_metrics()

        # Print epoch summary
        print(f'\nEpoch {epoch+1} Summary:')
        print(f'Generator Loss: {epoch_g_loss:.4f}')
        print(f'Discriminator Loss: {epoch_d_loss:.4f}')
        print(f'Classifier A - Loss: {epoch_cls_a_loss:.4f}, Acc: {epoch_cls_a_acc:.4f}')
        print(f'Classifier B - Loss: {epoch_cls_b_loss:.4f}, Acc: {epoch_cls_b_acc:.4f}')

        # Memory management
        gc.collect()
        torch.cuda.empty_cache()

    total_time = time.time() - start_time
    print(f'\nTraining completed in {total_time/3600:.2f} hours')
    return metrics_tracker

def main():
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Data transforms
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Create full dataset
    full_dataset = RetinalDataset('/content/dataset/train', transform=transform)

    # Split dataset (70% train, 30% test)
    train_size = int(0.7 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

    # Show and save transformed samples
    show_transformed_samples(full_dataset)

    # Initialize models and move to device
    generator = Generator().to(device)
    discriminator = Discriminator().to(device)
    classifier_a = create_classifier().to(device)
    classifier_b = create_classifier().to(device)

    # Initialize optimizers
    optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.9, 0.999))
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.9, 0.999))
    optimizer_cls_a = optim.Adam(classifier_a.parameters(), lr=0.001)
    optimizer_cls_b = optim.Adam(classifier_b.parameters(), lr=0.001)

    # Loss functions
    criterion_gan = nn.BCELoss()
    criterion_content = nn.MSELoss()
    criterion_cls = nn.CrossEntropyLoss()

    # Group models and optimizers
    models = (generator, discriminator, classifier_a, classifier_b)
    optimizers = (optimizer_g, optimizer_d, optimizer_cls_a, optimizer_cls_b)
    criteria = (criterion_gan, criterion_content, criterion_cls)

    # Train models
    metrics_tracker = train_models(
        train_loader, test_loader, models, optimizers, criteria, device,
        num_epochs=150, save_interval=10
    )

    # Final evaluation
    evaluate_models(test_loader, models, device)


main()

Using device: cpu


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/150: 100%|██████████| 10/10 [01:43<00:00, 10.39s/it, G_loss=0.91, D_loss=0.629, ClsA_loss=0.128, ClsB_loss=3.35]



Epoch 1 Summary:
Generator Loss: 1.8307
Discriminator Loss: 0.6431
Classifier A - Loss: 0.7108, Acc: 0.7881
Classifier B - Loss: 1.3670, Acc: 0.6026


Epoch 2/150: 100%|██████████| 10/10 [01:32<00:00,  9.28s/it, G_loss=0.767, D_loss=0.64, ClsA_loss=0.0973, ClsB_loss=0.646]



Epoch 2 Summary:
Generator Loss: 0.8433
Discriminator Loss: 0.6422
Classifier A - Loss: 0.3981, Acc: 0.8874
Classifier B - Loss: 0.7617, Acc: 0.6424


...


Epoch 148 Summary:
Generator Loss: 0.4169
Discriminator Loss: 0.0055
Classifier A - Loss: 0.0001, Acc: 1.0000
Classifier B - Loss: 0.0000, Acc: 1.0000


Epoch 149/150: 100%|██████████| 10/10 [01:34<00:00,  9.42s/it, G_loss=0.447, D_loss=0.00525, ClsA_loss=0.000269, ClsB_loss=0.000109]



Epoch 149 Summary:
Generator Loss: 0.4180
Discriminator Loss: 0.0050
Classifier A - Loss: 0.0001, Acc: 1.0000
Classifier B - Loss: 0.0003, Acc: 1.0000


Epoch 150/150: 100%|██████████| 10/10 [01:34<00:00,  9.44s/it, G_loss=0.384, D_loss=0.00475, ClsA_loss=0.000178, ClsB_loss=0.000246]



Epoch 150 Summary:
Generator Loss: 0.4144
Discriminator Loss: 0.0052
Classifier A - Loss: 0.0002, Acc: 1.0000
Classifier B - Loss: 0.0002, Acc: 1.0000

Training completed in 3.97 hours
